In [1]:
import pymongo
import pandas as pd
import time
import datetime
import xlwings as xw
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數
client = pymongo.MongoClient()
db = client['Fields']

In [2]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\季資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)


path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\年資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Y processing.. ', k)
    Y = v.drop('股票名稱', axis=1).set_index('股票代號')
    Y.columns = [str(int(s[:4]) + 1) + '0331' for s in Y.columns]
    Y.columns = pd.to_datetime(Y.columns)
    Y = Y.T.dropna(how='all', axis=0)
    Y.columns = Y.columns.astype(str)
    Y['日期'] = Y.index
    update_mongo(Y, k)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\日資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('D processing.. ', k)
    D = v.drop('股票名稱', axis=1).set_index('股票代號')
    D.columns = [s[:8] for s in D.columns]
    D.columns = pd.to_datetime(D.columns)
    D = D.T.dropna(how='all', axis=0)
    D.columns = D.columns.astype(str)
    D['日期'] = D.index
    update_mongo(D, k)


processing..  負債總計_Q
processing..  資產總計_Q
processing..  營業收入淨額_Q
processing..  EPS_Q
processing..  預估EPS_Q
processing..  現金股利殖利率_Q
processing..  現金流量比率_Q
processing..  槓桿比率_Q
processing..  EBITDA_Q
processing..  存貨週轉天數_Q
processing..  存貨_Q
processing..  現金及約當現金_Q
processing..  折舊費用_Q
processing..  不動產廠房設備_Q
